In [4]:
import json
from os import environ

from dotenv import load_dotenv

In [5]:
load_dotenv()

ESRI_USERNAME = environ.get("ESRI_USERNAME")
ESRI_PASSWORD = environ.get("ESRI_PASSWORD")
ESRI_REFERER = environ.get("ESRI_REFERER")

In [6]:
from ladb_data.service import LadbDataService

service = LadbDataService(ESRI_USERNAME, ESRI_PASSWORD, ESRI_REFERER)

In [7]:
# rows = service.get_road_segments_by_name("Tedman Street")
rows = service.get_road_segments_by_road_id("QLDRTED1530483827353171000")

https://qportal.information.qld.gov.au/arcgis/rest/services/LOC/Queensland_Roads_and_Tracks/FeatureServer/0/query?where=road_id%3D+%27QLDRTED1530483827353171000%27&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=%22road_id%22%2C%22segment_id%22%2C%22road_name_full%22%2C%22alias_1_name_full%22%2C%22lga_name_left%22%2C%22lga_name_right%22%2C%22locality_right%22%2C%22locality_left%22&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&returnTrueCurves=false&returnExceededLimitFeatures=false&quantizationParameters=&returnCentroid=false&timeReferenceUnknownClient=false&sqlFormat=none&result

In [8]:
print(json.dumps(rows[0], indent=2))

{
  "road_id": "QLDRTED1530483827353171000",
  "segment_id": "16015103990000359191",
  "road_name_full": "Tedman Street",
  "alias_1_name_full": null,
  "lga_name_left": "Brisbane City",
  "lga_name_right": "Brisbane City",
  "locality_right": "Zillmere",
  "locality_left": "Zillmere",
  "geometry_paths": [
    [
      [
        153.04854996000006,
        -27.352295009999978
      ],
      [
        153.04850604000012,
        -27.352440989999934
      ]
    ]
  ]
}


In [9]:
rows.append({
    "road_id": "QLDRTED1530483827353171000",
    "parent_id": "16015103990000359196",
    "segment_id": "16015103990000359196-0",
    "road_name_full": "Tedman Street",
    "alias_1_name_full": None,
    "lga_name_left": "Brisbane City",
    "lga_name_right": "Brisbane City",
    "locality_right": "Zillmere",
    "locality_left": "Zillmere",
    "geometry_paths": [
      [
        [
          153.04835088000004,
          -27.35335196999995
        ],
        [
          153.04814604000012,
          -27.354465989999937
        ]
      ]
    ]
  })

In [10]:
# 1. Filter for top-level road segments and add the triple (road-object sdo:hasPart road-segment) and its inverse sdo:isPartOf
#    - Top-level road segments do not have parent_id field
# 2. For each top-level road segment, add it to the graph
# #. Recurse down using depth-first search for each road segment where the parent_id is the segment_id of the parent.

from rdflib import Graph, SDO, URIRef, Literal

from ladb_data.models.road import RoadObject, SirIdentifier, ROAD

graph = Graph()

for row in rows:
    road_iri = f"https://linked.data.gov.au/dataset/qld-addr/road-{row['road_id']}"
    graph.add((URIRef(road_iri), SDO.identifier, Literal(row['road_id'], datatype="urn:ladb:sir-id")))
    id_ = f"https://linked.data.gov.au/dataset/qld-addr/road-segment-{row['segment_id']}"
    is_part_of_id = f"https://linked.data.gov.au/dataset/qld-addr/road-segment-{row['parent_id']}" if row.get("parent_id") else road_iri

    segment = RoadObject(
        id=id_,
        sir_identifier=SirIdentifier(value=row['segment_id']),
        type=[str(ROAD.RoadSegment), str(ROAD.RoadObject)],
        label=None,
        is_part_of=is_part_of_id
    )
    graph += segment.model_dump_graph()

for s, p, o in graph.triples((None, SDO.isPartOf, None)):
    graph.add((o, SDO.hasPart, s))

In [11]:
graph.print()

@prefix schema: <https://schema.org/> .

<https://linked.data.gov.au/dataset/qld-addr/road-segment-16015103990000359191> a <https://linked.data.gov.au/def/roads/RoadObject>,
        <https://linked.data.gov.au/def/roads/RoadSegment> ;
    schema:identifier "16015103990000359191"^^<urn:ladb:sir-id> ;
    schema:isPartOf <https://linked.data.gov.au/dataset/qld-addr/road-QLDRTED1530483827353171000> .

<https://linked.data.gov.au/dataset/qld-addr/road-segment-16015103990000359192> a <https://linked.data.gov.au/def/roads/RoadObject>,
        <https://linked.data.gov.au/def/roads/RoadSegment> ;
    schema:identifier "16015103990000359192"^^<urn:ladb:sir-id> ;
    schema:isPartOf <https://linked.data.gov.au/dataset/qld-addr/road-QLDRTED1530483827353171000> .

<https://linked.data.gov.au/dataset/qld-addr/road-segment-16015103990000359193> a <https://linked.data.gov.au/def/roads/RoadObject>,
        <https://linked.data.gov.au/def/roads/RoadSegment> ;
    schema:identifier "16015103990000359193